In [107]:
import pandas as pd

In [108]:
RIDERS = pd.read_csv("./docs/data/riders.csv")
RIDER_TEAMS = pd.read_csv("./docs/data/rider_teams.csv")
MANAGERS = pd.read_csv("./docs/data/managers.csv")
MANAGER_TEAMS = pd.read_csv("./docs/data/manager_teams.csv")
POINTS_SYSTEM = pd.read_csv("./docs/data/points_system.csv")
RACES = pd.read_csv("./docs/data/races.csv")
RESULTS_2023 = pd.read_csv("./docs/data/results/results_2023_full.csv")
RESULTS_2024 = pd.read_csv("./docs/data/results/results_2024_full.csv")
RESULTS_2025 = pd.read_csv("./docs/data/results/results_2025_full.csv")

In [140]:
# Riders owned by
RIDERS_OWNED_BY = MANAGER_TEAMS.groupby(["RiderName"])["ManagerName"].apply(", ".join).reset_index()

# Race categories
RACE_CATEGORIES = RACES[["RaceName_PCS", "RaceCategory"]]

# Relevant results
RESULTS_POINTS = RESULTS_2025[["RACE", "STAGE_ID", "STAGE_TYPE", "RANK", "RIDER"]]
RESULTS_POINTS = RESULTS_POINTS.rename(columns = {
    "RACE": "RaceName",
    "STAGE_ID": "StageID",
    "STAGE_TYPE": "StageType",
    "RANK": "Rank",
    "RIDER": "RiderName"
})
RESULTS_POINTS = RESULTS_POINTS.dropna(subset = ["StageID"])
RESULTS_POINTS = RESULTS_POINTS.loc[RESULTS_POINTS["StageType"] != "TTT"]

def convert_to_int(row):
    if row.isdigit():
        return int(row)
    else:
        return 0
    
def stage_or_gcresult(stage_id, race_category):
    if stage_id.startswith("stage"):
        return race_category.replace("Tour ", "Stage ")
    else:
        return race_category    

RESULTS_POINTS = pd.merge(RESULTS_POINTS, RACE_CATEGORIES, how = "inner", left_on = ["RaceName"], right_on = ["RaceName_PCS"])
RESULTS_POINTS["Rank"] = RESULTS_POINTS.apply(lambda row: convert_to_int(row["Rank"]), axis = 1)
RESULTS_POINTS["RaceCategory"] = RESULTS_POINTS.apply(lambda row: stage_or_gcresult(row["StageID"], row["RaceCategory"]), axis = 1)
RESULTS_POINTS = pd.merge(RESULTS_POINTS, POINTS_SYSTEM, how = "left", left_on = ["RaceCategory", "Rank"], right_on = ["RaceCategory", "RaceRank"])
RESULTS_POINTS = pd.merge(RESULTS_POINTS, RIDERS, how = "left", left_on = ["RiderName"], right_on = ["RiderName_PCS"])

# Rider points
RIDER_POINTS = RESULTS_POINTS.groupby(["RiderName_Zweeler"])["RacePoints"].sum().reset_index()

# Rider racedays
RIDER_RACEDAYS = RESULTS_POINTS.loc[RESULTS_POINTS["StageID"] != "gc"]
RIDER_RACEDAYS = RIDER_RACEDAYS.groupby(["RiderName_Zweeler"])["RiderName_Zweeler"].count().rename("Racedays").reset_index()
#RIDER_RACEDAYS.rename(columns = {0: "RiderName_Zweeler", 1: "Racedays"}, inplace = True)

In [ ]:
RIDER_OUTPUT = RIDERS

RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, RIDER_RACEDAYS, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName_Zweeler"])
RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, RIDER_POINTS, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName_Zweeler"])
RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, RIDERS_OWNED_BY, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName"])



RIDER_OUTPUT["PPM"] = 

# CREATE TABLE CONSISTING OF:
# RIDERNAME
# RIDERTEAM
# RACEDAYS TOTAL (Number from PCS)
# RACEDAYS IN GAME (NUmber of stages incl. DNF/DNS, excluding TTT and GC)
# Points 2025 (calculated)
# Points 2024 (calculated)
# PPM 2025 (calculated)
# Picked by number in game (need to scrape from Zweeler)
# Cheapo eligible (<= 2.5 + not banned)
# Managers owning

   RiderName_Zweeler     RiderName_PCS                    RiderTeam  \
0    Aberasturi, Jon    Aberasturi Jon          Euskaltel - Euskadi   
1  Abrahamsen, Jonas  Abrahamsen Jonas               Uno-X Mobility   
2  Ackermann, Pascal  Ackermann Pascal        Israel - Premier Tech   
3       Adrià, Roger       Adrià Roger  Red Bull - BORA - hansgrohe   
4     Aert, Wout van     Van Aert Wout    Team Visma - Lease a Bike   

   RiderPrice  Racedays  RacePoints          RiderName  \
0         2.7       NaN         NaN                NaN   
1         6.2       5.0         0.0  Abrahamsen, Jonas   
2         6.6       NaN         NaN                NaN   
3         6.8       NaN         NaN       Adrià, Roger   
4        37.4       NaN         NaN     Aert, Wout van   

                            ManagerName  
0                                   NaN  
1                                 Tommy  
2                                   NaN  
3                                Okholm  
4  Chrelle, Ja

In [111]:
"""
BANS
Kenk: Staune-Mittet
Visti: Jeanniere
Jarma: Rex
Jappo: Tim Tom Timtomberg
Chrelle: Withen
Tore: Goossens
Hustlersen: Kogut
Kenk: Aleotti
Visti: Seixas
Jarma: Uhlig
Jappo: Biermans
Chrelle: Brenner
Tore: Behrens
Hustlersen: Sam Watson
"""

'\nBANS\nKenk: Staune-Mittet\nVisti: Jeanniere\nJarma: Rex\nJappo: Tim Tom Timtomberg\nChrelle: Withen\nTore: Goossens\nHustlersen: Kogut\nKenk: Aleotti\nVisti: Seixas\nJarma: Uhlig\nJappo: Biermans\nChrelle: Brenner\nTore: Behrens\nHustlersen: Sam Watson\n'